In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.utils import shuffle
from hashlib import md5
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [3]:
df = pd.read_csv("~\\kaggle\\input\\edgeiiotset-cyber-security-dataset-of-iot-iiot\\Edge-IIoTset dataset\Selected dataset for ML and DL\\DNN-EdgeIIoT-dataset.csv", low_memory = False)

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\aadit\AppData\Local\Temp\ipykernel_8424\3834739954.py:1: SyntaxWarning: invalid escape sequence '\S'
  df = pd.read_csv("~\\kaggle\\input\\edgeiiotset-cyber-security-dataset-of-iot-iiot\\Edge-IIoTset dataset\Selected dataset for ML and DL\\DNN-EdgeIIoT-dataset.csv", low_memory = False)


In [5]:
# Display all column names to find relevant protocol indicators
print(df.columns.tolist())

['frame.time', 'ip.src_host', 'ip.dst_host', 'arp.dst.proto_ipv4', 'arp.opcode', 'arp.hw.size', 'arp.src.proto_ipv4', 'icmp.checksum', 'icmp.seq_le', 'icmp.transmit_timestamp', 'icmp.unused', 'http.file_data', 'http.content_length', 'http.request.uri.query', 'http.request.method', 'http.referer', 'http.request.full_uri', 'http.request.version', 'http.response', 'http.tls_port', 'tcp.ack', 'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack', 'tcp.dstport', 'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.options', 'tcp.payload', 'tcp.seq', 'tcp.srcport', 'udp.port', 'udp.stream', 'udp.time_delta', 'dns.qry.name', 'dns.qry.name.len', 'dns.qry.qu', 'dns.qry.type', 'dns.retransmission', 'dns.retransmit_request', 'dns.retransmit_request_in', 'mqtt.conack.flags', 'mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as', 'mqtt.msg', 'mqtt.msgtype', 'mqtt.proto_len', 'mqtt.protoname', 'mqtt.t

In [6]:
# Select application layer protocol columns
app_layer_cols = ['http.file_data', 'http.content_length', 'http.request.uri.query', 'http.request.method', 'http.referer', 'http.request.full_uri', 'http.request.version', 'http.response', 'http.tls_port', 'dns.qry.name', 'dns.qry.name.len', 'dns.qry.qu', 'dns.qry.type', 'dns.retransmission', 'dns.retransmit_request', 'dns.retransmit_request_in', 'mqtt.conack.flags', 'mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as', 'mqtt.msg', 'mqtt.msgtype', 'mqtt.proto_len', 'mqtt.protoname', 'mqtt.topic', 'mqtt.topic_len', 'mqtt.ver', 'mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id']


# Replace NaNs with 0 for protocol columns (if needed)
df[app_layer_cols] = df[app_layer_cols].fillna(0)

# Ensure binary 0/1 format
df[app_layer_cols] = df[app_layer_cols].applymap(lambda x: 1 if x != 0 else 0)

# Keep rows where any application-layer protocol is present
df = df[df[app_layer_cols].sum(axis=1) > 0]

print(f"Remaining rows after application-layer filtering: {len(df)}")


C:\Users\aadit\AppData\Local\Temp\ipykernel_8424\3696147179.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[app_layer_cols] = df[app_layer_cols].applymap(lambda x: 1 if x != 0 else 0)


Remaining rows after application-layer filtering: 2219201


In [7]:
df = df.rename(columns={'http.request.method': 'http1', 'http.referer': 'http2', 'http.request.version': 'http3', 'dns.qry.name.len': 'dns', 'mqtt.conack.flags': 'mqtt1', 'mqtt.protoname': 'mqtt2', 'mqtt.topic': 'mqtt3'})

In [8]:
le_http1 = LabelEncoder()
le_http2 = LabelEncoder()
le_http3 = LabelEncoder()
le_dns = LabelEncoder()
le_mqtt1 = LabelEncoder()
le_mqtt2 = LabelEncoder()
le_mqtt3 = LabelEncoder()

In [9]:
df['http1_encoded'] = le_http1.fit_transform(df.http1)
df['http2_encoded'] = le_http2.fit_transform(df.http2)
df['http3_encoded'] = le_http3.fit_transform(df.http3)
df['dns_encoded'] = le_dns.fit_transform(df.dns)
df['mqtt1_encoded'] = le_mqtt1.fit_transform(df.mqtt1)
df['mqtt2_encoded'] = le_mqtt2.fit_transform(df.mqtt2)
df['mqtt3_encoded'] = le_mqtt3.fit_transform(df.mqtt3)

In [10]:
http1_ = OneHotEncoder()
http2_ = OneHotEncoder()
http3_ = OneHotEncoder()
dns_ = OneHotEncoder()
mqtt1_ = OneHotEncoder()
mqtt2_ = OneHotEncoder()
mqtt3_ = OneHotEncoder()

In [11]:
X1 = http1_.fit_transform(df.http1_encoded.values.reshape(-1,1)).toarray()
X2 = http2_.fit_transform(df.http2_encoded.values.reshape(-1,1)).toarray()
X3 = http3_.fit_transform(df.http3_encoded.values.reshape(-1,1)).toarray()
X4 = dns_.fit_transform(df.dns_encoded.values.reshape(-1,1)).toarray()
X5 = mqtt1_.fit_transform(df.mqtt1_encoded.values.reshape(-1,1)).toarray()
X6 = mqtt2_.fit_transform(df.mqtt2_encoded.values.reshape(-1,1)).toarray()
X7 = mqtt3_.fit_transform(df.mqtt3_encoded.values.reshape(-1,1)).toarray()

In [12]:
dfOneHot = pd.DataFrame(X1, columns = ["http1_"+str(int(i)) for i in range(X1.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(X2, columns = ["http2_"+str(int(i)) for i in range(X2.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(X3, columns = ["http3_"+str(int(i)) for i in range(X3.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(X4, columns = ["dns_"+str(int(i)) for i in range(X4.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(X5, columns = ["mqtt1_"+str(int(i)) for i in range(X5.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(X6, columns = ["mqtt2_"+str(int(i)) for i in range(X6.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)
dfOneHot = pd.DataFrame(X7, columns = ["mqtt3_"+str(int(i)) for i in range(X7.shape[1])])
df = pd.concat([df, dfOneHot], axis=1)

In [13]:
df.drop(columns=['http1','http2','http3', 'dns','mqtt1','mqtt2', 'mqtt3'], inplace = True )
df.shape

(2219201, 70)

In [14]:
df.isnull().sum()

frame.time            0
ip.src_host           0
ip.dst_host           0
arp.dst.proto_ipv4    0
arp.opcode            0
                     ..
http3_0               0
dns_0                 0
mqtt1_0               0
mqtt2_0               0
mqtt3_0               0
Length: 70, dtype: int64

In [15]:
df.duplicated().sum()

np.int64(819)

In [16]:
df = df.drop_duplicates()

In [17]:
# Function to create a hash for each column
def hash_column(series):
    return md5(pd.util.hash_pandas_object(series, index=False).values).hexdigest()

# Function to find columns with identical hashes
def find_identical_columns_by_hash(df):
    hash_dict = {}
    for col in df.columns:
        col_hash = hash_column(df[col])
        if col_hash in hash_dict:
            hash_dict[col_hash].append(col)
        else:
            hash_dict[col_hash] = [col]

    return [cols for cols in hash_dict.values() if len(cols) > 1]

# Applying the function to the DataFrame
identical_column_groups = find_identical_columns_by_hash(df)
print("Groups of identical columns:", identical_column_groups)

Groups of identical columns: [['icmp.unused', 'http.tls_port', 'dns.qry.type', 'mqtt.msg_decoded_as', 'http1_encoded', 'http2_encoded', 'http3_encoded', 'dns_encoded', 'mqtt1_encoded', 'mqtt2_encoded', 'mqtt3_encoded'], ['http.file_data', 'http.request.uri.query', 'http.request.full_uri', 'mqtt.msg'], ['mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.proto_len', 'mqtt.ver'], ['mqtt.hdrflags', 'mqtt.msgtype'], ['mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id'], ['http1_0', 'http2_0', 'http3_0', 'dns_0', 'mqtt1_0', 'mqtt2_0', 'mqtt3_0']]


In [20]:
# Groups of identical columns
identical_column_groups = [
    ['icmp.unused', 'http.tls_port', 'dns.qry.type', 'mqtt.msg_decoded_as', 'http1_encoded', 'http2_encoded', 'http3_encoded', 'dns_encoded', 'mqtt1_encoded', 'mqtt2_encoded', 'mqtt3_encoded'], 
    ['http.file_data', 'http.request.uri.query', 'http.request.full_uri', 'mqtt.msg'], 
    ['mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.proto_len', 'mqtt.ver'], 
    ['mqtt.hdrflags', 'mqtt.msgtype'], 
    ['mbtcp.len', 'mbtcp.trans_id', 'mbtcp.unit_id'], 
    ['http1_0', 'http2_0', 'http3_0', 'dns_0', 'mqtt1_0', 'mqtt2_0', 'mqtt3_0']
]

# Iterate through the list of groups and drop all but the first column
for group in identical_column_groups:
    # Keep the first column of the group and drop the rest
    columns_to_drop = group[1:]  # all columns except the first one
    df = df.drop(columns_to_drop, axis=1, errors='ignore')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2218382 entries, 0 to 2219200
Data columns (total 45 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   frame.time                 object 
 1   ip.src_host                object 
 2   ip.dst_host                object 
 3   arp.dst.proto_ipv4         object 
 4   arp.opcode                 float64
 5   arp.hw.size                float64
 6   arp.src.proto_ipv4         object 
 7   icmp.checksum              float64
 8   icmp.seq_le                float64
 9   icmp.transmit_timestamp    float64
 10  icmp.unused                float64
 11  http.file_data             int64  
 12  http.content_length        int64  
 13  http.response              int64  
 14  tcp.ack                    float64
 15  tcp.ack_raw                float64
 16  tcp.checksum               float64
 17  tcp.connection.fin         float64
 18  tcp.connection.rst         float64
 19  tcp.connection.syn         float64
 20  tcp.con

In [22]:
print(df['Attack_type'].value_counts())

Attack_type
Normal                   1615643
DDoS_UDP                  121567
DDoS_ICMP                 116436
SQL_injection              51203
Password                   50153
Vulnerability_scanner      50110
DDoS_TCP                   50062
DDoS_HTTP                  49911
Uploading                  37634
Backdoor                   24862
Port_Scanning              22564
XSS                        15915
Ransomware                 10925
Fingerprinting              1001
MITM                         396
Name: count, dtype: int64


In [23]:
print(df['Attack_type'].value_counts())

Attack_type
Normal                   1615643
DDoS_UDP                  121567
DDoS_ICMP                 116436
SQL_injection              51203
Password                   50153
Vulnerability_scanner      50110
DDoS_TCP                   50062
DDoS_HTTP                  49911
Uploading                  37634
Backdoor                   24862
Port_Scanning              22564
XSS                        15915
Ransomware                 10925
Fingerprinting              1001
MITM                         396
Name: count, dtype: int64


In [24]:
df_cat = df.select_dtypes(exclude=[np.number])
df_cat.describe(include='all')

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.src.proto_ipv4,tcp.options,tcp.payload,tcp.srcport,Attack_type
count,2218382,2218382,2218382,2218382,2218382,2218382,2218382,2218382,2218382
unique,2206364,137167,52425,9,10,242565,274925,61975,15
top,192.168.0.128,192.168.0.128,192.168.0.128,0,0,0,0,1883.0,Normal
freq,11533,962129,958578,2200599,2090565,1453980,1300742,664514,1615643


In [26]:
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.dst.proto_ipv4", "arp.src.proto_ipv4",
                "http.file_data", "http.request.uri.query",
                "http.request.full_uri", "tcp.options", "tcp.payload",
                "tcp.srcport", "mqtt.msg"]
df.drop(drop_columns, axis = 1, inplace = True, errors='ignore')
df.dropna(axis = 0, how = 'any', inplace = True)
df.drop_duplicates(subset = None, keep = "first", inplace = True)
df = shuffle(df)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1944298 entries, 2207535 to 898765
Data columns (total 36 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   arp.opcode                 float64
 1   arp.hw.size                float64
 2   icmp.checksum              float64
 3   icmp.seq_le                float64
 4   icmp.transmit_timestamp    float64
 5   icmp.unused                float64
 6   http.content_length        int64  
 7   http.response              int64  
 8   tcp.ack                    float64
 9   tcp.ack_raw                float64
 10  tcp.checksum               float64
 11  tcp.connection.fin         float64
 12  tcp.connection.rst         float64
 13  tcp.connection.syn         float64
 14  tcp.connection.synack      float64
 15  tcp.dstport                float64
 16  tcp.flags                  float64
 17  tcp.flags.ack              float64
 18  tcp.len                    float64
 19  tcp.seq                    float64
 20  ud

In [28]:
print(df['Attack_type'].value_counts())

Attack_type
Normal                   1398480
DDoS_UDP                  121567
DDoS_ICMP                  67939
SQL_injection              50826
DDoS_TCP                   50062
Vulnerability_scanner      50026
Password                   49933
DDoS_HTTP                  48544
Uploading                  36957
Backdoor                   24026
Port_Scanning              19977
XSS                        15068
Ransomware                  9689
Fingerprinting               853
MITM                         351
Name: count, dtype: int64


In [29]:
columns_to_drop = ['icmp.unused']
df = df.drop(columns=columns_to_drop)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1944298 entries, 2207535 to 898765
Data columns (total 35 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   arp.opcode                 float64
 1   arp.hw.size                float64
 2   icmp.checksum              float64
 3   icmp.seq_le                float64
 4   icmp.transmit_timestamp    float64
 5   http.content_length        int64  
 6   http.response              int64  
 7   tcp.ack                    float64
 8   tcp.ack_raw                float64
 9   tcp.checksum               float64
 10  tcp.connection.fin         float64
 11  tcp.connection.rst         float64
 12  tcp.connection.syn         float64
 13  tcp.connection.synack      float64
 14  tcp.dstport                float64
 15  tcp.flags                  float64
 16  tcp.flags.ack              float64
 17  tcp.len                    float64
 18  tcp.seq                    float64
 19  udp.port                   float64
 20  ud

In [31]:
print(df['Attack_type'].value_counts())

Attack_type
Normal                   1398480
DDoS_UDP                  121567
DDoS_ICMP                  67939
SQL_injection              50826
DDoS_TCP                   50062
Vulnerability_scanner      50026
Password                   49933
DDoS_HTTP                  48544
Uploading                  36957
Backdoor                   24026
Port_Scanning              19977
XSS                        15068
Ransomware                  9689
Fingerprinting               853
MITM                         351
Name: count, dtype: int64


In [32]:
# Your DataFrame is named 'data' in this example
df.to_csv('Edge-IIoTset_112.csv', index=False)